In [ ]:
################################################################################
## Test code for primal-dual IP                                               ##
################################################################################
using JuMP
using GLPK
using Random
using LinearAlgebra
using ForwardDiff
using Plots

In [ ]:
################################################################################
## Find an initial strictly feasible primal solution                          ##
################################################################################
function strict_primal(c::Vector{Float64}, A::Matrix{Float64}, b::Vector{Float64})

    (m, n) = size(A)

    lp = Model(with_optimizer(GLPK.Optimizer))

    @variable(lp, t >= 0)
    @variable(lp, x[1:n] >= 0)
    @constraint(lp, A*x .== b)
    @constraint(lp, [i = 1:n], t >= 1 - x[i])
    @objective(lp, Min, t)

    optimize!(lp)

    return (value(t), value.(x))
end

## Update Newton direction
function update_newton_dir(c::Vector{Float64}, A::Matrix{Float64}, b::Vector{Float64},
                           x::Vector{Float64}, u::Vector{Float64}, v::Vector{Float64},
                           μ::Float64)
    
    ## Diagonalize u and x
    U = Diagonal(u)
    X = Diagonal(x)
    e = ones(length(x))
    
    ## Primal and dual residuals
    rp = A*x - b
    rd = A'*v + u - c - Q*x  
    
    #### TODO: Derive and write the Newton update formulas 
    dᵥ = 
    dₓ = 
    dᵤ = 
   
    return (dᵥ,dᵤ,dₓ)
end

################################################################################
## Calculate step size that retains feasibility                             ##
################################################################################
function calculate_step_size(x::Vector{Float64}, d::Vector{Float64}, ϵ::Float64)
    n = length(d)
    α = 0.9999
    for i = 1:n
        if d[i] < 0
            α = min(α, -x[i]/d[i]) # prevents variable becoming negative
        end
    end
    return round(α, digits = Int(-log10(ϵ))) #rounding avoids numerical issues
end


################################################################################
## Solve the QP problem and its dual                                          ##
##                                                                            ##
## minimize    cᵀx + (1/2)xᵀQx        maximize    bᵀv - (1/2)xᵀQx             ##
## subject to  Ax = b                 subject to  Aᵀv + u - Qx = c            ##
##              x ≥ 0                                        u ≥ 0            ##
##                                                                            ##
##                                                                            ##
##                                                                            ##
################################################################################
function primal_dual_ip(A::Matrix{Float64}, b::Vector{Float64}, c::Vector{Float64},
                        Q::Matrix{Float64}, β::Float64, ϵ::Float64, N::Int, μ::Float64)

    n = length(c)    # Number of primal variables x and dual variables u
    m = length(b)    # Number of dual variables v
    x = zeros(N, n)  # Primal variable x values
    u = zeros(N, n)  # Dual variable u values
    v = zeros(N, m)  # Dual variable v values
    e = ones(n)      # Vector of ones
    α_p = 1          # Step size for variable x update
    α_d = 1          # Step size for variable u, and v update

    ## Find an initial solution
    (⋅, x₀) = strict_primal(c, A, b)
    u₀ = μ ./ x₀

    u[1,:] = u₀
    x[1,:] = x₀      # Note v₀ initialised as 0

    ## Main loop
    for i = 1:N
        ## Stopping condition #1
        if n*μ < ϵ           ## if dot(c,x[i,:]) - dot(b,v[i,:]) < ϵ
            v = v[1:i,:]
            u = u[1:i,:]
            x = x[1:i,:]
            return (v,u,x)
        end
        ## Update dₓ, dᵥ, dᵤ
        (dᵥ,dᵤ,dₓ) = update_newton_dir(c, A, b, x[i,:], u[i,:], v[i,:], μ)

        ## Calculate step size α primal and α dual
        α_p = calculate_step_size(x[i,:], dₓ, ϵ)
        α_d = calculate_step_size(u[i,:], dᵤ, ϵ)

        ## Update variables
        v[i+1,:] = v[i,:] + α_d*dᵥ
        u[i+1,:] = u[i,:] + α_d*dᵤ
        x[i+1,:] = x[i,:] + α_p*dₓ

        ## Stopping condition #2
        if i == N-1
            return (v,u,x)
        end
        ## Update μ
        μ = μ*β
    end
end

Random.seed!(1)

## Problem data in standard form
c = [-1.0,-1.0, 0.0,0.0,0.0,0.0,0.0]
b = [ 5.0,-1.0,-8.0,9.0,4.0]
A = [-1/3  1.0 1.0 0.0 0.0 0.0 0.0;
      1/5 -1.0 0.0 1.0 0.0 0.0 0.0;
     -8/3 -1.0 0.0 0.0 1.0 0.0 0.0;
      1/2  1.0 0.0 0.0 0.0 1.0 0.0;
      1.0 -1.0 0.0 0.0 0.0 0.0 1.0]

## Create random PD matrix Q for the quadratic term
n = length(c)
Q = randn(n, n)                # Create random matrix
Q = (Q + Q')/2                 # Make A symmetric
if isposdef(A) == false        # Check if A is PD
    λmin = eigmin(Q)           # Minimum eigenvalue
    Q = Q + (abs(λmin) + 1)*I  # Add λmin + 1 to diagonal elements
end
Q[3:n,:] .= 0.0                 # Set zero values for slack variables
Q[:,3:n] .= 0.0

## Standard form:
# min      -x₁ - x₂ + ⁠(1/2)xᵀQx
# s.t.: -1/3x₁ + x₂ + x₃                 =  5
#        1/5x₁ - x₂     + x₄             = -1
#       -8/3x₁ - x₂         + x₅         = -8
#        1/2x₁ + x₂             + x₆     =  9
#           x₁ - x₂ +               + x₇ =  4
#           x₁, ... ,x₇ ≧  0


## Executions and plotting
N = 100     # Number of iterations
β = 0.3     # Reduction factor
μ = 10.0    # Penalty parameter
ϵ = 1e-6    # Tolerance

## Solution time
@time (v,u,x) = primal_dual_ip(A, b, c, Q, β, ϵ, N, μ)

## Plotting
ng = 1000
x1 = LinRange(0, 15, ng)
x2 = LinRange(0, 15, ng)

plot(x1, 5 .+ (1/3).*x1,  color = :blue, reuse = false, legend = false)
plot!(x1, 1 .+ (1/5).*x1, color = :blue)
plot!(x1, 8 .- (8/3).*x1, color = :blue)
plot!(x1, 9 .- (1/2).*x1, color = :blue)
plot!(x1, -4 .+ x1, color = :blue,
      xaxis  = ("x1", (0,10)),
      yaxis  = ("x2", (0,10)),
      aspect_ratio = :equal,
      size   = (800,800),
      legend = false,
      title  = "IPM path")

g(x) = dot(c[1:2],x) + (1/2)*dot(x,Q[1:2,1:2]*x)
z = [g([x1[i],x2[j]]) for j = 1:ng, i = 1:ng]

contour!(x1, x2, z,
        levels  = [2, 9.27, 27, 64, 128, 256],
        clims   = (0,300),
        clabels = true)

traj = x[1:end,1:2]
f(x) = x[1] + x[2]

plot!(x1, f(traj[end,:]) .- x1, color = :2, line = :dash)
annotate!([(traj[1,1] + 0.2, traj[1,2] + 0.5,
            text("x0",9,:bottom, RGBA(0.25,0.65,0.30,1.0)))])

display(plot!(traj[:,1], traj[:,2], marker = :o))
savefig("qp_convergence.pdf")